In [9]:
import utils
import pandas as pd
import expert_factory
import numpy as np

In [81]:
#Lendo o dadoa
df = pd.read_csv('data/treinamento-1.txt', header=None)
num_lags = 10

#criando Lag
lagged_data = utils.create_lag(df, num_lags)
lagged_data = lagged_data.reset_index(drop=True)

X = lagged_data.drop(['y'],axis=1)
y = lagged_data['y']

#Criando conjunto de dados
fracao_dados_para_treino = 0.7
fracao_dados_para_teste = 0.2
X_train,y_train,X_test,y_test,X_val,y_val = utils.treino_teste_validacao(X,y, frac_train=fracao_dados_para_treino, frac_test=fracao_dados_para_teste)

Tamanho total 990
Tamanho treino 693
Tamanho teste 198
Tamanho validacao 99


In [12]:
#vamos comecar com experts iguais
exp_ne= X_train.shape[1]
exp_nh= 3
exp_ns= 1
num_experts = 2
all_experts = []
for _ in range(num_experts):
    exp = expert_factory.Expert(exp_ne,exp_nh,exp_ns,g_h='sigmoid',g_o='sigmoid')
    all_experts.append(exp)
    
gating_ne = X_train.shape[1]
gating_nh = 3
gating_ns = num_experts
gating_network = expert_factory.Expert(gating_ne,gating_nh,gating_ns, g_h='sigmoid', g_o='softmax')


In [ ]:
likelihood = 0
old_likelihood = -np.inf
iters = 0
max_iters = 10
matriz_covariancia = np.identity(nExperts)
while abs(likelihood-old_likelihood) > 1e-3 and iters < max_iters:
    iters += 1
    #calcula a saida para cada rede
    #A funcao retorna 4 varaiveis, queremos apenas a ultima que representa o output (por isso o [-1])
    output_gating = gating_network.feedforward(X_train)[-1]
    output_experts = np.matrix([np.ravel(expert.feedforward(X_train)[-1]).tolist() for expert in all_experts]).T
    
    #Agora que temos a saida comecamos com a funcao de EM
    
    #Passo E (Expectation)
    #Com os parametros atuais calculamos calculamos a 'expectation' posterior para cada expert
    old_likelihood = likelihood
    likelihood_train,haux_train = calcula_verossimilhanca(num_experts, y_train, output_experts, output_gating, matriz_covariancia)
    
    #likelihood_val,haux_val = calcula_verossimilhanca(num_experts, y_train, output_experts, output_gating, matriz_covariancia)
    
    #h = haux./(sum(haux,2)*ones(1,m));
    h = np.divide(haux_train, np.sum(haux_train,axis=1))
    
    #Passo M (Maximizacao)
    

In [2]:
import numpy as np
import pandas as pd
import math
import time
import matplotlib.pyplot as plt
import itertools
import sys

#import Perceptron_Regressor
#import Utilities

import importlib
# importlib.reload(Perceptron_Regressor)
# importlib.reload(Utilities)


In [121]:
def calcula_verossimilhanca(num_experts, y_true, output_experts, output_gating, matriz_covariancia):
    all_py = []
    for expert in range(num_experts):
        row = []
        for index,y in enumerate(y_true):
            #calcula a diff entre o real e o que o cada expert previu
            diff = y - output_experts[index,expert]
            #Py(j,i)=exp(-diff*diff'/(2*var(i)));   
            py = np.exp(np.dot(-diff, diff.T) / (2 * matriz_covariancia[expert,expert]))
            row.append(py)
        all_py.append(row)
    
    all_py = np.matrix(all_py).T
    # Likelihood= sum(log(sum(Yg.*Py,2)));
    #haux = Yg.*Py;
    haux = np.multiply(output_gating, all_py)
    likelihood = np.sum(np.log(np.sum(aux,axis=1)))
    return likelihood,haux
        
           
        

In [119]:
def calc_likelihood(d, y_experts, y_g, covar_matrix):
    #list with the probablities of all errors from each expert to each line of input
    py_matrix = []
    #iterate over the experts
    for exp in range(y_experts[0].shape[1]):
        py_row = []
        #for each expert, calculate the individual py (sum of logs of final output - the gating output multiplied by the corresponding expert output)
        for inst, value in enumerate(d):
            #diff is the error of the final output
            diff = value-y_experts[inst,exp]
            py = np.exp(-np.dot(diff, diff.T) / np.multiply(2, covar_matrix[exp,exp]))
            py_row += [py[0,0]]
        py_matrix += [py_row]
    
    py_matrix = np.matrix(py_matrix).T
    h_aux = np.multiply(y_g, py_matrix)
    likelihood = np.sum(np.log(np.sum(h_aux, axis=1)))
    
    return likelihood, h_aux

In [ ]:
#util = Utilities.Utilities()

# def calc_likelihood(d, y_experts, y_g, covar_matrix):
#     #create the matrix with the errors of each expert to each instance
#     diff = d-y_experts
    
#     #create the auxiliar matrix that computes the squared error for each expert,
#     # multiplied by the covariance matrix,
#     # Note: in this case we use the Identity for the covariance matrix
#     expert_error = np.sum(np.multiply(covar_matrix, np.dot(diff.T, diff)), axis=0)

#     # calculates py as the prior probability multiplied by the exp 
#     # of the expert_error multiplied by -0.5
#     py = np.multiply(np.exp(np.multiply(expert_error, -0.5)), y_g)

#     #calculates the final likelihood, computing the sum for each instance of the logs of py
#     ll = -np.sum(np.log(py))
    
#     return ll, py

def calc_2norm(m):
    return np.sqrt(np.sum(np.multiply(m,m)))

def maximize_gating(gating_net, X_train, h, max_it = 1e4, min_norm = 1e-5, X_valid=None, d_valid=None, use_fit=False):
    if use_fit:
        gating_net.fit(X=X_train, d=h, valid_data=X_valid, valid_d=d_valid, verbose=False)
    else:
        norm_grad = float("inf")
        it = 0

        while norm_grad > min_norm and it < max_it:
            print(it, norm_grad)
            #calculate the descent gradient for h
            djdw1, djdw2 = gating_net.calculate_gradient(X=X_train, d=h)
            #compute the right leraning rate
            learn_rate = gating_net.calculate_bisection(X=X_train, d=h, djdw1=djdw1, djdw2=djdw2)
            #learn_rate=0.1
            #update the gating network wheights
            gating_net.w1, gating_net.w2 = gating_net.update_weights(learning_rate=learn_rate, djdw1=djdw1, djdw2=djdw2,
                                                                     w1=gating_net.w1, w2=gating_net.w2)
            it+=1
            norm_grad = calc_2norm(np.append(djdw1.ravel(), djdw2.ravel(), axis=1))

def maximize_expert(expert_net, X_train, h, d_train, covar_matrix=None, max_it = 1e4, min_norm = 1e-5, X_valid=None, d_valid=None, use_fit=False):
    if use_fit:
        expert_net.fit(X=X_train, d=d_train, valid_data=X_valid, valid_d=d_valid, side_factor=h, verbose=False)
    else:
        norm_grad = float("inf")
        it = 0

        #print("expert start new")
        while norm_grad > min_norm and it < max_it:
            #calculate the descent gradient for h
            #print("expert start")
            djdw1, djdw2 = gating_net.calculate_gradient(X=X_train, d=d_train, side_factor=h)
            #print("grad")
            #compute the right leraning rate
            learn_rate = gating_net.calculate_bisection(X=X_train, d=d_train, djdw1=djdw1, djdw2=djdw2, side_factor=h)
            #learn_rate=0.1
            #print("alfa")
            #update the gating network wheights
            gating_net.w1, gating_net.w2 = gating_net.update_weights(learning_rate=learn_rate, djdw1=djdw1, djdw2=djdw2,
                                                                     w1=gating_net.w1, w2=gating_net.w2)

            it+=1
            norm_grad = calc_2norm(np.append(djdw1.ravel(), djdw2.ravel(), axis=1))

            #print("weights")

def calc_final_pred(X, gating_net, experts_list):
    y_g = gating_net.forward(X)

    #y_e = []
    #for exp in experts_list:
    #    y_e +=[exp.forward(X).T[0]]
    #    print()
    #print(y_e)
    #y_e =np.matrix(y_e).T
    y_e = np.matrix([np.array(exp.forward(X).T.tolist()[0]) for exp in experts_list]).T

    return np.sum(np.multiply(y_e, y_g), axis=1)


def main():

    # In[ ]:
    input_path = r"input\treinamento-1.txt"

    input_data=pd.read_csv(open(input_path, "r"), header=None)
    input_data.columns = ["time_series"]
    input_data.head()


    # In[17]:


    lagged_data = util.create_lags(input_data, n=20)

    #util.to_excel(lagged_data, "time_series_with_lag.xlsx")


    # In[18]:


    correlations = lagged_data.corr()

    #util.to_excel(correlations, "correlations.xlsx")


    # In[19]:


    # CRIA OS INPUTS
    max_lags = 5

    lagged_data_ = lagged_data[["time_series"] + ["time_series_lag_{}".format(i) for i in range(1, max_lags+1)]].dropna()
    test, train = util.get_simple_sample(lagged_data_, 0.7)
    valid, test = util.get_simple_sample(test, 0.5)

    X_train= train[["time_series_lag_{}".format(i) for i in range(1, max_lags+1)]]
    X_train["bias"] = 1 # adicionando bias
    d_train = train[["time_series"]]

    X_test = test[["time_series_lag_{}".format(i) for i in range(1, max_lags+1)]]
    X_test["bias"] = 1 # adicionando bias
    d_test = test[["time_series"]]

    X_valid = valid[["time_series_lag_{}".format(i) for i in range(1, max_lags+1)]]
    X_valid["bias"] = 1 # adicionando bias
    d_valid = valid[["time_series"]]

    nInp = len(X_train.loc[0])
    nOut = len(d_train.loc[0])
    nHid_gat = 3
    nHid_exp = 3
    nExperts = 4

    #creates the gating network
    gating_net = Perceptron_Regressor.MLP(nInp=nInp, nHid=nHid_gat, nOut=nExperts,
                                          fFunc="sigmoid", gFunc="softmax",
                                          cost_func="entropy")

    #creates the expert networks list
    experts_list = []
    for i in range(nExperts):
        expert = Perceptron_Regressor.MLP(nInp=nInp, nHid=nHid_gat, nOut=nOut,
                                          fFunc="sigmoid", gFunc="sigmoid",
                                          cost_func="mse")
        
        experts_list+=[expert]


    likelihood = 0
    likelihood_prev = -float("inf")
    max_iterations = 1000
    min_ll_gain = 1e-3
    covar_matrix = np.identity(nExperts)

    #loop to execute the Expectation Maximization algorithm
    it = 0
    while it < max_iterations and abs(likelihood-likelihood_prev) > min_ll_gain:
    #     st_time = time.time()
        it+=1
        #calculates the outputs of each network
        y_g = gating_net.forward(X_train)
        #y_experts = np.matrix([exp.forward(X_train).T[0] for exp in experts_list]).T
        y_experts = np.matrix([np.array(exp.forward(X_train).T.tolist()[0]) for exp in experts_list]).T
        
        y_g_valid = gating_net.forward(X_valid)
        #y_experts_valid = np.matrix([exp.forward(X_valid).T[0] for exp in experts_list]).T
        y_experts_valid = np.matrix([np.array(exp.forward(X_valid).T.tolist()[0]) for exp in experts_list]).T
        
        #E step - Expectation
        # calculates the matrix h of posterior expectations for each expert
        likelihood_prev = likelihood
        likelihood, h_aux = calc_likelihood(d=np.matrix(d_train), y_experts=y_experts,
                                            y_g=y_g, covar_matrix=covar_matrix)
        likelihood_valid, h_aux_valid = calc_likelihood(d=np.matrix(d_valid), y_experts=y_experts_valid,
                                            y_g=y_g_valid, covar_matrix=covar_matrix)
    #     print("Likelihood time =", time.time()-st_time)
        
    #     st_time = time.time()
        #computes the h (posteriori likelihood) dividing elementwise the h_aux
        # by the sum of all elements of the matrix 
        h = np.divide(h_aux, np.sum(h_aux, axis=1))
        h_valid = np.divide(h_aux_valid, np.sum(h_aux_valid, axis=1))
        
        #M step - Maximization
        # minimize the cost function for gating and expert networks (maximize the ouputs)
        #First - maximize gating network (calulate the descend gradient for the error to h)
        maximize_gating(gating_net=gating_net, X_train=X_train, h=h,
                        X_valid=X_valid, d_valid=d_valid,use_fit=True)
    #     print("Maximixing Gating time =", time.time()-st_time)
        
        #then maximize each of the experts
        for k, expert in enumerate(experts_list):
    #         st_time = time.time()
            #compute the expert responsability in the error for each instance
            #expert_responsability = np.multiply(d_train, np.divide(np.sum(h, axis=0)[0,k], covar_matrix[k,k]))        
            maximize_expert(expert_net=expert, X_train=X_train, h=h[:,k],
                            d_train=d_train, covar_matrix=covar_matrix,
                            X_valid=X_valid, d_valid=d_valid,use_fit=True)
    #         print("\t\tMaximazing Expert",k,"time =", time.time()-st_time)
            
        y_pred = calc_final_pred(X=X_train, gating_net=gating_net, experts_list=experts_list)
        mse = util.get_mse(y_pred, d_train)
        print(it, "\t", likelihood, "\t", mse)
    print("y_pred:\n", y_pred)

if __name__ == "__main__":
    main()